In [1]:
import argparse
import os
import sys
import datetime
import time
import math
import json
from pathlib import Path

import numpy as np
from PIL import Image
import torch
import torch.nn as nn
import torch.distributed as dist
import torch.backends.cudnn as cudnn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torchvision import models as torchvision_models

import utils
import vision_transformer as vits
from vision_transformer import DINOHead

import easydict
import torchvision
from tqdm import tqdm

In [2]:
torchvision_archs = sorted(name for name in torchvision_models.__dict__
    if name.islower() and not name.startswith("__")
    and callable(torchvision_models.__dict__[name]))

In [3]:
print(torchvision_archs)

['alexnet', 'convnext_base', 'convnext_large', 'convnext_small', 'convnext_tiny', 'densenet121', 'densenet161', 'densenet169', 'densenet201', 'efficientnet_b0', 'efficientnet_b1', 'efficientnet_b2', 'efficientnet_b3', 'efficientnet_b4', 'efficientnet_b5', 'efficientnet_b6', 'efficientnet_b7', 'efficientnet_v2_l', 'efficientnet_v2_m', 'efficientnet_v2_s', 'get_weight', 'googlenet', 'inception_v3', 'mnasnet0_5', 'mnasnet0_75', 'mnasnet1_0', 'mnasnet1_3', 'mobilenet_v2', 'mobilenet_v3_large', 'mobilenet_v3_small', 'regnet_x_16gf', 'regnet_x_1_6gf', 'regnet_x_32gf', 'regnet_x_3_2gf', 'regnet_x_400mf', 'regnet_x_800mf', 'regnet_x_8gf', 'regnet_y_128gf', 'regnet_y_16gf', 'regnet_y_1_6gf', 'regnet_y_32gf', 'regnet_y_3_2gf', 'regnet_y_400mf', 'regnet_y_800mf', 'regnet_y_8gf', 'resnet101', 'resnet152', 'resnet18', 'resnet34', 'resnet50', 'resnext101_32x8d', 'resnext101_64x4d', 'resnext50_32x4d', 'shufflenet_v2_x0_5', 'shufflenet_v2_x1_0', 'shufflenet_v2_x1_5', 'shufflenet_v2_x2_0', 'squeezenet1

In [4]:
# def get_args_parser():
#     parser = argparse.ArgumentParser('DINO', add_help=False)

#     # Model parameters
#     parser.add_argument('--arch', default='vit_small', type=str,
#         choices=['vit_tiny', 'vit_small', 'vit_base', 'xcit', 'deit_tiny', 'deit_small'] \
#                 + torchvision_archs + torch.hub.list("facebookresearch/xcit:main"),
#         help="""Name of architecture to train. For quick experiments with ViTs,
#         we recommend using vit_tiny or vit_small.""")
#     parser.add_argument('--patch_size', default=16, type=int, help="""Size in pixels
#         of input square patches - default 16 (for 16x16 patches). Using smaller
#         values leads to better performance but requires more memory. Applies only
#         for ViTs (vit_tiny, vit_small and vit_base). If <16, we recommend disabling
#         mixed precision training (--use_fp16 false) to avoid unstabilities.""")
#     parser.add_argument('--out_dim', default=65536, type=int, help="""Dimensionality of
#         the DINO head output. For complex and large datasets large values (like 65k) work well.""")
#     parser.add_argument('--norm_last_layer', default=True, type=utils.bool_flag,
#         help="""Whether or not to weight normalize the last layer of the DINO head.
#         Not normalizing leads to better performance but can make the training unstable.
#         In our experiments, we typically set this paramater to False with vit_small and True with vit_base.""")
#     parser.add_argument('--momentum_teacher', default=0.996, type=float, help="""Base EMA
#         parameter for teacher update. The value is increased to 1 during training with cosine schedule.
#         We recommend setting a higher value with small batches: for example use 0.9995 with batch size of 256.""")
#     parser.add_argument('--use_bn_in_head', default=False, type=utils.bool_flag,
#         help="Whether to use batch normalizations in projection head (Default: False)")

#     # Temperature teacher parameters
#     parser.add_argument('--warmup_teacher_temp', default=0.04, type=float,
#         help="""Initial value for the teacher temperature: 0.04 works well in most cases.
#         Try decreasing it if the training loss does not decrease.""")
#     parser.add_argument('--teacher_temp', default=0.04, type=float, help="""Final value (after linear warmup)
#         of the teacher temperature. For most experiments, anything above 0.07 is unstable. We recommend
#         starting with the default value of 0.04 and increase this slightly if needed.""")
#     parser.add_argument('--warmup_teacher_temp_epochs', default=0, type=int,
#         help='Number of warmup epochs for the teacher temperature (Default: 30).')

#     # Training/Optimization parameters
#     parser.add_argument('--use_fp16', type=utils.bool_flag, default=True, help="""Whether or not
#         to use half precision for training. Improves training time and memory requirements,
#         but can provoke instability and slight decay of performance. We recommend disabling
#         mixed precision if the loss is unstable, if reducing the patch size or if training with bigger ViTs.""")
#     parser.add_argument('--weight_decay', type=float, default=0.04, help="""Initial value of the
#         weight decay. With ViT, a smaller value at the beginning of training works well.""")
#     parser.add_argument('--weight_decay_end', type=float, default=0.4, help="""Final value of the
#         weight decay. We use a cosine schedule for WD and using a larger decay by
#         the end of training improves performance for ViTs.""")
#     parser.add_argument('--clip_grad', type=float, default=3.0, help="""Maximal parameter
#         gradient norm if using gradient clipping. Clipping with norm .3 ~ 1.0 can
#         help optimization for larger ViT architectures. 0 for disabling.""")
#     parser.add_argument('--batch_size_per_gpu', default=64, type=int,
#         help='Per-GPU batch-size : number of distinct images loaded on one GPU.')
#     parser.add_argument('--epochs', default=100, type=int, help='Number of epochs of training.')
#     parser.add_argument('--freeze_last_layer', default=1, type=int, help="""Number of epochs
#         during which we keep the output layer fixed. Typically doing so during
#         the first epoch helps training. Try increasing this value if the loss does not decrease.""")
#     parser.add_argument("--lr", default=0.0005, type=float, help="""Learning rate at the end of
#         linear warmup (highest LR used during training). The learning rate is linearly scaled
#         with the batch size, and specified here for a reference batch size of 256.""")
#     parser.add_argument("--warmup_epochs", default=10, type=int,
#         help="Number of epochs for the linear learning-rate warm up.")
#     parser.add_argument('--min_lr', type=float, default=1e-6, help="""Target LR at the
#         end of optimization. We use a cosine LR schedule with linear warmup.""")
#     parser.add_argument('--optimizer', default='adamw', type=str,
#         choices=['adamw', 'sgd', 'lars'], help="""Type of optimizer. We recommend using adamw with ViTs.""")
#     parser.add_argument('--drop_path_rate', type=float, default=0.1, help="stochastic depth rate")

#     # Multi-crop parameters
#     parser.add_argument('--global_crops_scale', type=float, nargs='+', default=(0.4, 1.),
#         help="""Scale range of the cropped image before resizing, relatively to the origin image.
#         Used for large global view cropping. When disabling multi-crop (--local_crops_number 0), we
#         recommand using a wider range of scale ("--global_crops_scale 0.14 1." for example)""")
#     parser.add_argument('--local_crops_number', type=int, default=8, help="""Number of small
#         local views to generate. Set this parameter to 0 to disable multi-crop training.
#         When disabling multi-crop we recommend to use "--global_crops_scale 0.14 1." """)
#     parser.add_argument('--local_crops_scale', type=float, nargs='+', default=(0.05, 0.4),
#         help="""Scale range of the cropped image before resizing, relatively to the origin image.
#         Used for small local view cropping of multi-crop.""")

#     # Misc
#     parser.add_argument('--data_path', default='/path/to/imagenet/train/', type=str,
#         help='Please specify path to the ImageNet training data.')
#     parser.add_argument('--output_dir', default=".", type=str, help='Path to save logs and checkpoints.')
#     parser.add_argument('--saveckp_freq', default=20, type=int, help='Save checkpoint every x epochs.')
#     parser.add_argument('--seed', default=0, type=int, help='Random seed.')
#     parser.add_argument('--num_workers', default=10, type=int, help='Number of data loading workers per GPU.')
#     parser.add_argument("--dist_url", default="env://", type=str, help="""url used to set up
#         distributed training; see https://pytorch.org/docs/stable/distributed.html""")
#     parser.add_argument("--local_rank", default=0, type=int, help="Please ignore and do not set this argument.")
#     return parser

In [5]:
pwd

'C:\\Users\\dhkim\\Desktop\\directory\\dino-main'

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
args = easydict.EasyDict({
     # Model parameters
    "arch" : 'vit_small',
    'patch_size' : 16,
    'out_dim' : 65536,
    'norm_last_layer' : True,
    'momentum_teacher' : 0.996,
    'use_bn_in_head' : False,
    # Temperature teacher parameters
    'warmup_teacher_temp' : 0.04,
    'teacher_temp' : 0.04,
    'warmup_teacher_temp_epochs' : 0,
    # Training/Optimization parameters
    'use_fp16' : True,
    'weight_decay' : 0.04,
    'weight_decay_end' : 0.4,
    'clip_grad' : 3.0,
    'batch_size_per_gpu' : 64,
    'epochs' : 100,
    'freeze_last_layer' : 1,
    'lr' : 0.0005,
    'warmup_epochs' : 10,
    'min_lr' : 1e-6,
    'optimizer' : 'adamw',
    'drop_path_rate' : 0.1,
    # Multi-crop parameters
    'global_crops_scale': (0.4, 1.),
    'local_crops_number' : 8,
    'local_crops_scale' : (0.05, 0.4),
     # Misc
    'data_path' : 'C:\\Users\\dhkim\\Desktop\\directory\\imagenet_data',
    'output_dir' : 'C:\\Users\\dhkim\\Desktop\\directory\\dino-main/main_pth',
    'saveckp_freq' : 20,
    'seed' : 0,
    'num_workers' : 0,
#     'dist_url' : 'env://'
    'local_rank' : 0,
    'gpu' : device
   })

In [13]:
def train_one_epoch(student, teacher, teacher_without_ddp, dino_loss, data_loader,
                    optimizer, lr_schedule, wd_schedule, momentum_schedule,epoch,
                    fp16_scaler, args):
    metric_logger = utils.MetricLogger(delimiter="  ")
    header = 'Epoch: [{}/{}]'.format(epoch, args.epochs)
    print("header")
    print(header)
    

#     print(data_loader)
    
    for it, (images, _) in tqdm(enumerate(data_loader)):
#         print(it)
        if it % 10 == 0:
            print(f"이터레이션: {it}, 메트릭 정보: {metric_logger}")
            
        # update weight decay and learning rate according to their schedule
        
        
        it = len(data_loader) * epoch + it  # global training iteration
        for i, param_group in enumerate(optimizer.param_groups):
            param_group["lr"] = lr_schedule[it]
            if i == 0:  # only the first group is regularized
                param_group["weight_decay"] = wd_schedule[it]
            
#         print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")
        
        # move images to gpu
        images = [im.cuda(non_blocking=True) for im in images]
        # teacher and student forward passes + compute dino loss
        with torch.cuda.amp.autocast(fp16_scaler is not None):
            teacher_output = teacher(images[:2])  # only the 2 global views pass through the teacher
            student_output = student(images)
            loss = dino_loss(student_output, teacher_output, epoch)

        if not math.isfinite(loss.item()):
            print("Loss is {}, stopping training".format(loss.item()), force=True)
            sys.exit(1)

        # student update
        optimizer.zero_grad()
        param_norms = None
        if fp16_scaler is None:
            loss.backward()
            if args.clip_grad:
                param_norms = utils.clip_gradients(student, args.clip_grad)
            utils.cancel_gradients_last_layer(epoch, student,
                                              args.freeze_last_layer)
            optimizer.step()
        else:
            fp16_scaler.scale(loss).backward()
            if args.clip_grad:
                fp16_scaler.unscale_(optimizer)  # unscale the gradients of optimizer's assigned params in-place
                param_norms = utils.clip_gradients(student, args.clip_grad)
            utils.cancel_gradients_last_layer(epoch, student,
                                              args.freeze_last_layer)
            fp16_scaler.step(optimizer)
            fp16_scaler.update()

        # EMA update for the teacher
        with torch.no_grad():
            m = momentum_schedule[it]  # momentum parameter
            for param_q, param_k in zip(student.parameters(), teacher_without_ddp.parameters()):
                param_k.data.mul_(m).add_((1 - m) * param_q.detach().data)

        # logging
        torch.cuda.synchronize()
        metric_logger.update(loss=loss.item())
        metric_logger.update(lr=optimizer.param_groups[0]["lr"])
        metric_logger.update(wd=optimizer.param_groups[0]["weight_decay"])
    # gather the stats from all processes
    metric_logger.synchronize_between_processes()
    print("Averaged stats:", metric_logger)
    return {k: meter.global_avg for k, meter in metric_logger.meters.items()}

In [14]:
def train_dino(args):
#     utils.init_distributed_mode(args)
    utils.fix_random_seeds(args.seed)
    print("git:\n  {}\n".format(utils.get_sha()))
    print("\n".join("%s: %s" % (k, str(v)) for k, v in sorted(dict(vars(args)).items())))
    cudnn.benchmark = True

    # ============ preparing data ... ============
    transform = DataAugmentationDINO(
        args.global_crops_scale,
        args.local_crops_scale,
        args.local_crops_number,
    )
    dataset = torchvision.datasets.CIFAR100(root='../data', train=True,
                                    download=True, transform=transform)
    
    data_loader = torch.utils.data.DataLoader(
        dataset,
        batch_size=args.batch_size_per_gpu,
        num_workers=args.num_workers,
        pin_memory=True,
        drop_last=True,
    )
    print(f"Data loaded: there are {len(dataset)} images.")

    # ============ building student and teacher networks ... ============
    # we changed the name DeiT-S for ViT-S to avoid confusions
    args.arch = args.arch.replace("deit", "vit")
    # if the network is a Vision Transformer (i.e. vit_tiny, vit_small, vit_base)
    if args.arch in vits.__dict__.keys():
        student = vits.__dict__[args.arch](
            patch_size=args.patch_size,
            drop_path_rate=args.drop_path_rate,  # stochastic depth
        )
        teacher = vits.__dict__[args.arch](patch_size=args.patch_size)
        embed_dim = student.embed_dim
    # if the network is a XCiT
    elif args.arch in torch.hub.list("facebookresearch/xcit:main"):
        student = torch.hub.load('facebookresearch/xcit:main', args.arch,
                                 pretrained=False, drop_path_rate=args.drop_path_rate)
        teacher = torch.hub.load('facebookresearch/xcit:main', args.arch, pretrained=False)
        embed_dim = student.embed_dim
    # otherwise, we check if the architecture is in torchvision models
    elif args.arch in torchvision_models.__dict__.keys():
        student = torchvision_models.__dict__[args.arch]()
        teacher = torchvision_models.__dict__[args.arch]()
        embed_dim = student.fc.weight.shape[1]
    else:
        print(f"Unknow architecture: {args.arch}")

    # multi-crop wrapper handles forward with inputs of different resolutions
    student = utils.MultiCropWrapper(student, DINOHead(
        embed_dim,
        args.out_dim,
        use_bn=args.use_bn_in_head,
        norm_last_layer=args.norm_last_layer,
    ))
    teacher = utils.MultiCropWrapper(
        teacher,
        DINOHead(embed_dim, args.out_dim, args.use_bn_in_head),
    )
    # move networks to gpu
    student, teacher = student.cuda(), teacher.cuda()
    # synchronize batch norms (if any)
    if utils.has_batchnorms(student):
        student = nn.SyncBatchNorm.convert_sync_batchnorm(student)
        teacher = nn.SyncBatchNorm.convert_sync_batchnorm(teacher)

        # we need DDP wrapper to have synchro batch norms working...
        teacher = nn.parallel.DistributedDataParallel(teacher, device_ids=[args.gpu])
        teacher_without_ddp = teacher.module
    else:
        # teacher_without_ddp and teacher are the same thing
        teacher_without_ddp = teacher
    student.to(args.gpu)
    # teacher and student start with the same weights
    teacher_without_ddp.load_state_dict(student.state_dict())
    # there is no backpropagation through the teacher, so no need for gradients
    for p in teacher.parameters():
        p.requires_grad = False
    print(f"Student and Teacher are built: they are both {args.arch} network.")

    # ============ preparing loss ... ============
    dino_loss = DINOLoss(
        args.out_dim,
        args.local_crops_number + 2,  # total number of crops = 2 global crops + local_crops_number
        args.warmup_teacher_temp,
        args.teacher_temp,
        args.warmup_teacher_temp_epochs,
        args.epochs,
    ).cuda()

    # ============ preparing optimizer ... ============
    params_groups = utils.get_params_groups(student)
    if args.optimizer == "adamw":
        optimizer = torch.optim.AdamW(params_groups)  # to use with ViTs
    elif args.optimizer == "sgd":
        optimizer = torch.optim.SGD(params_groups, lr=0, momentum=0.9)  # lr is set by scheduler
    elif args.optimizer == "lars":
        optimizer = utils.LARS(params_groups)  # to use with convnet and large batches
    # for mixed precision training
    fp16_scaler = None
    if args.use_fp16:
        fp16_scaler = torch.cuda.amp.GradScaler()

    # ============ init schedulers ... ============
    lr_schedule = utils.cosine_scheduler(
        args.lr * (args.batch_size_per_gpu * utils.get_world_size()) / 256.,  # linear scaling rule
        args.min_lr,
        args.epochs, len(data_loader),
        warmup_epochs=args.warmup_epochs,
    )
    wd_schedule = utils.cosine_scheduler(
        args.weight_decay,
        args.weight_decay_end,
        args.epochs, len(data_loader),
    )
    # momentum parameter is increased to 1. during training with a cosine schedule
    momentum_schedule = utils.cosine_scheduler(args.momentum_teacher, 1,
                                               args.epochs, len(data_loader))
    print(f"Loss, optimizer and schedulers ready.")

    # ============ optionally resume training ... ============
    to_restore = {"epoch": 0}
    utils.restart_from_checkpoint(
        os.path.join(args.output_dir, "checkpoint.pth"),
        run_variables=to_restore,
        student=student,
        teacher=teacher,
        optimizer=optimizer,
        fp16_scaler=fp16_scaler,
        dino_loss=dino_loss,
    )
    start_epoch = to_restore["epoch"]

    start_time = time.time()
    print("Starting DINO training !")
    for epoch in tqdm(range(start_epoch, args.epochs)):
#         data_loader.sampler.set_epoch(epoch)

        # ============ training one epoch of DINO ... ============
        train_stats = train_one_epoch(student, teacher, teacher_without_ddp, dino_loss,
            data_loader, optimizer, lr_schedule, wd_schedule, momentum_schedule,
            epoch, fp16_scaler, args)
        print("!!!!!!!!!!!!!!!!!")
        
        # ============ writing logs ... ============
        save_dict = {
            'student': student.state_dict(),
            'teacher': teacher.state_dict(),
            'optimizer': optimizer.state_dict(),
            'epoch': epoch + 1,
            'args': args,
            'dino_loss': dino_loss.state_dict(),
        }
        if fp16_scaler is not None:
            save_dict['fp16_scaler'] = fp16_scaler.state_dict()
        utils.save_on_master(save_dict, os.path.join(args.output_dir, 'checkpoint.pth'))
        if args.saveckp_freq and epoch % args.saveckp_freq == 0:
            utils.save_on_master(save_dict, os.path.join(args.output_dir, f'checkpoint{epoch:04}.pth'))
        log_stats = {**{f'train_{k}': v for k, v in train_stats.items()},
                     'epoch': epoch}
        if utils.is_main_process():
            with (Path(args.output_dir) / "log.txt").open("a") as f:
                f.write(json.dumps(log_stats) + "\n")
    total_time = time.time() - start_time
    total_time_str = str(datetime.timedelta(seconds=int(total_time)))
    print('Training time {}'.format(total_time_str))

In [15]:
class DINOLoss(nn.Module):
    def __init__(self, out_dim, ncrops, warmup_teacher_temp, teacher_temp,
                 warmup_teacher_temp_epochs, nepochs, student_temp=0.1,
                 center_momentum=0.9):
        super().__init__()
        self.student_temp = student_temp
        self.center_momentum = center_momentum
        self.ncrops = ncrops
        self.register_buffer("center", torch.zeros(1, out_dim))
        # we apply a warm up for the teacher temperature because
        # a too high temperature makes the training instable at the beginning
        self.teacher_temp_schedule = np.concatenate((
            np.linspace(warmup_teacher_temp,
                        teacher_temp, warmup_teacher_temp_epochs),
            np.ones(nepochs - warmup_teacher_temp_epochs) * teacher_temp
        ))

    def forward(self, student_output, teacher_output, epoch):
        """
        Cross-entropy between softmax outputs of the teacher and student networks.
        """
        student_out = student_output / self.student_temp
        student_out = student_out.chunk(self.ncrops)

        # teacher centering and sharpening
        temp = self.teacher_temp_schedule[epoch]
        teacher_out = F.softmax((teacher_output - self.center) / temp, dim=-1)
        teacher_out = teacher_out.detach().chunk(2)

        total_loss = 0
        n_loss_terms = 0
        for iq, q in enumerate(teacher_out):
            for v in range(len(student_out)):
                if v == iq:
                    # we skip cases where student and teacher operate on the same view
                    continue
                loss = torch.sum(-q * F.log_softmax(student_out[v], dim=-1), dim=-1)
                total_loss += loss.mean()
                n_loss_terms += 1
        total_loss /= n_loss_terms
        self.update_center(teacher_output)
        return total_loss

    @torch.no_grad()
    def update_center(self, teacher_output):
        """
        Update center used for teacher output.
        """
        batch_center = torch.sum(teacher_output, dim=0, keepdim=True)
#         dist.all_reduce(batch_center)
        batch_center = batch_center / (len(teacher_output))

        # ema update
        self.center = self.center * self.center_momentum + batch_center * (1 - self.center_momentum)

In [16]:
class DataAugmentationDINO(object):
    def __init__(self, global_crops_scale, local_crops_scale, local_crops_number):
        flip_and_color_jitter = transforms.Compose([
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.RandomApply(
                [transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.2, hue=0.1)],
                p=0.8
            ),
            transforms.RandomGrayscale(p=0.2),
        ])
        normalize = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
        ])

        # first global crop
        self.global_transfo1 = transforms.Compose([
            transforms.RandomResizedCrop(224, scale=global_crops_scale, interpolation=Image.BICUBIC),
            flip_and_color_jitter,
            utils.GaussianBlur(1.0),
            normalize,
        ])
        # second global crop
        self.global_transfo2 = transforms.Compose([
            transforms.RandomResizedCrop(224, scale=global_crops_scale, interpolation=Image.BICUBIC),
            flip_and_color_jitter,
            utils.GaussianBlur(0.1),
            utils.Solarization(0.2),
            normalize,
        ])
        # transformation for the local small crops
        self.local_crops_number = local_crops_number
        self.local_transfo = transforms.Compose([
            transforms.RandomResizedCrop(96, scale=local_crops_scale, interpolation=Image.BICUBIC),
            flip_and_color_jitter,
            utils.GaussianBlur(p=0.5),
            normalize,
        ])

    def __call__(self, image):
        crops = []
        crops.append(self.global_transfo1(image))
        crops.append(self.global_transfo2(image))
        for _ in range(self.local_crops_number):
            crops.append(self.local_transfo(image))
        return crops

In [ ]:
train_dino(args)

git:
  sha: N/A, status: clean, branch: N/A

arch: vit_small
batch_size_per_gpu: 64
clip_grad: 3.0
data_path: C:\Users\dhkim\Desktop\directory\imagenet_data
drop_path_rate: 0.1
epochs: 100
freeze_last_layer: 1
global_crops_scale: [0.4, 1.0]
gpu: cuda
local_crops_number: 8
local_crops_scale: [0.05, 0.4]
local_rank: 0
lr: 0.0005
min_lr: 1e-06
momentum_teacher: 0.996
norm_last_layer: True
num_workers: 0
optimizer: adamw
out_dim: 65536
output_dir: C:\Users\dhkim\Desktop\directory\dino-main
patch_size: 16
saveckp_freq: 20
seed: 0
teacher_temp: 0.04
use_bn_in_head: False
use_fp16: True
warmup_epochs: 10
warmup_teacher_temp: 0.04
warmup_teacher_temp_epochs: 0
weight_decay: 0.04
weight_decay_end: 0.4
Files already downloaded and verified
Data loaded: there are 50000 images.
Student and Teacher are built: they are both vit_small network.
Loss, optimizer and schedulers ready.
Starting DINO training !


  0%|                                                                                          | 0/100 [00:00<?, ?it/s]

header
Epoch: [0/100]



0it [00:00, ?it/s]

이터레이션: 0, 메트릭 정보: 


...


161it [07:06,  2.71s/it]
162it [07:09,  2.76s/it]
163it [07:12,  2.78s/it]
164it [07:15,  2.79s/it]
165it [07:18,  2.78s/it]
166it [07:20,  2.73s/it]
167it [07:23,  2.68s/it]
168it [07:25,  2.66s/it]
169it [07:28,  2.71s/it]
170it [07:31,  2.73s/it]

이터레이션: 170, 메트릭 정보: loss: 5.236877 (5.235893)  lr: 0.000092 (0.000092)  wd: 0.170888 (0.170370)



171it [07:34,  2.76s/it]
172it [07:36,  2.68s/it]
173it [07:39,  2.63s/it]
174it [07:41,  2.61s/it]
175it [07:44,  2.60s/it]
176it [07:47,  2.66s/it]
177it [07:49,  2.66s/it]
178it [07:52,  2.70s/it]
179it [07:55,  2.66s/it]
180it [07:58,  2.71s/it]

이터레이션: 180, 메트릭 정보: loss: 5.155676 (5.229031)  lr: 0.000092 (0.000092)  wd: 0.170958 (0.170404)



181it [08:00,  2.68s/it]
182it [08:03,  2.63s/it]
183it [08:05,  2.64s/it]
184it [08:08,  2.66s/it]
185it [08:11,  2.67s/it]
186it [08:13,  2.64s/it]
187it [08:16,  2.70s/it]

In [ ]:
pwd

In [ ]:
# mkdir imagenet_data